# ...Capital One Data Challenge

## 2 . IRL - In Real Life

*Ideal-Data is an oxymoron.

DATA is IRRELEVANT, INCOMPLETE and INCORRECT.  
First, we FIND it.  
Then FILTER, FILL and FIX.  
Finally, FANCY.

In [ ]:
import pandas as pd, pandasql, uszipcode, pandas_profiling, plotly, plotly.graph_objs as go, qgrid, plotly_express as px

In [ ]:
# Read Data
zillow_df = pd.read_csv('Zip_Zhvi_2bedroom.csv')
airbnb_df = pd.read_csv('listings.csv',low_memory=False)

#### EDA - Exploratory Data Analysis

In [ ]:
# Check Dimensions
print('zillow :', zillow_df.shape)
print('airbnb :', airbnb_df.shape)

In [ ]:
pd.set_option('display.max_colwidth', 20)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.reset_option('display.max_colwidth')

In [ ]:
zillow_df.sample(5)

In [ ]:
zillow_df.describe()

In [ ]:
airbnb_df.sample(5)

In [ ]:
airbnb_df.describe()

#### Clean zillow data

In [ ]:
# FIND relevant columns with missing (INCOMPLETE) values
print ('Column        Missing')
zillow_df.isnull().sum()

In [ ]:
# Make sure 'City' column is clean
zillow_df[['City','RegionID']].groupby('City').count()

#### TODO : Machine learning model to predict current (2019) home cost from time-series data.

__FOR NOW*__ : Use latest available data (2017)  
Housing market can be volatile and using previous years to extrapolate may lead to worse data.

In [ ]:
# FILTER out data that is not needed (IRRELEVANT) for analysis
# Relevant columns = ZipCode, Cost
# Relevant rows = New York city

z_cols = ['RegionName', '2017-06']
zillow = zillow_df.loc[ zillow_df['City'] == 'New York',
                        z_cols ]

In [ ]:
# Format datatypes
zillow.reset_index(drop=True,inplace=True)
zillow.rename(columns={'RegionName':'zipcode'}, inplace=True)
zillow['zipcode'] = zillow['zipcode'].astype(str)
zillow['2017-06'] = zillow['2017-06'].astype(int)

# Make sure 'ZipCode' and 'Cost' columns are clean
zillow[zillow['zipcode'].str.len() != 5]
zillow[zillow['2017-06'] < 0]

display(print(zillow.shape))
zillow

#### Clean airbnb data

In [ ]:
# FIND relevant columns with missing (INCOMPLETE) values
print ('Column        Missing')
airbnb_df.isnull().sum()

# ZipCode has 517 NULL values (Wait.. don't FIX yet, FILTERing will reduce it)

In [ ]:
# Make sure 'Bedrooms' column is clean
airbnb_df[['bedrooms','id']].groupby('bedrooms').count()

In [ ]:
# FILTER out data that is not needed (IRRELEVANT) for analysis
# Relevant data = ZipCode,Price,Availability,Reviews
# Relevant rows = 2-bedroom properties

a_cols = ['neighbourhood_cleansed','neighbourhood_group_cleansed','zipcode','latitude','longitude','price',
          'availability_30','availability_60','availability_90','availability_365','number_of_reviews','last_review']

airbnb = airbnb_df.loc[ airbnb_df['bedrooms'] == 2,
                        a_cols ]

In [ ]:
# Clean & Format datatypes
airbnb.reset_index(drop=True,inplace=True)
airbnb['price'] = airbnb['price'].str.replace('\$|,|\.00','').astype(int)
airbnb['last_review'] = pd.to_datetime(airbnb['last_review'])
airbnb['latitude'] = airbnb['latitude'].astype(float)
airbnb['longitude'] = airbnb['longitude'].astype(float)

# Make sure 'ZipCode' and 'Price' columns are clean
airbnb[airbnb['zipcode'].str.len() != 5] # No need to FIX those 8 records found earlier. All were filtered out.
airbnb[airbnb['price'] < 0]

display(print(airbnb.shape))
airbnb.sample(10)

In [ ]:
# FIND NULL (INCOMPLETE) zip codes
null_zip = airbnb.loc[airbnb['zipcode'].isnull(),['zipcode','latitude','longitude']]

# No need to FILL all those 517 records found earlier. Only 50 remain
print (null_zip.shape)

#### EXTERNAL DATA - US zip code database

In [ ]:
# FILL INCOMPLETE data
# search by latitude and longitude to get zip code
airbnb.loc[airbnb['zipcode'].isnull(),['zipcode']] = [uszipcode.SearchEngine().by_coordinates(lat,lon)[0].values()[0] 
                                                        for lat,lon in zip(null_zip['latitude'],null_zip['longitude'])]

# No NULL zip codes
airbnb[airbnb['zipcode'].isnull()]

#### TODO : Function to estimate availability of property, calculate yearly revenue, and increase in rent over time.

__FOR-NOW__ : Estimate simple average of 30-day, 60-day, 90-day & 365-day availability from calendar

In [ ]:
airbnb['availability_year'] = (airbnb['availability_30']*12 +
                               airbnb['availability_60']*6 +
                               airbnb['availability_90']*3 +
                               airbnb['availability_365'])/4

#### Visual EDA


In [ ]:
pandas_profiling.ProfileReport(zillow)

In [ ]:
pandas_profiling.ProfileReport(airbnb)

In [ ]:
# Individual datasets were cleaned
# Check if joining datasets causes loss of information
print ('zipcodes - zillow:',len(zillow['zipcode'].unique()),' , airbnb:',len(airbnb['zipcode'].unique()))

In [ ]:
query = """
select zipcode from zillow
INTERSECT
select zipcode from airbnb
"""

# 24 zip codes in common
pandasql.sqldf(query)

Out of 25, 24 zip codes from Zillow are in AirBnB.  
Can we try to get additional data from AirBnB regarding the 1 zip code ? Not worth effort.

Out of 171, only 24 zip codes from AirBnB are in Zillow.  
Can we try to get additional data from Zillow regarding the 147 zip codes ? __YES__

#### EXTERNAL DATA - https://www.zillow.com/market-report/08-19/6181/new-york-ny.xls?rt=7

Sheet 'Two Bed' contains median 2019 'List Price'  
It's a better estimate of 2019 home cost than 2017 cost, and it also additional zip code data


In [ ]:
zillow_list = pd.read_csv('zillow_listprice.csv')

In [ ]:
# Join new and old zillow datasets to FILL INCOMPLETE & FIX INCORRECT data
query = """
-- Use 2019 List Price. If the data doesn't exist, use 2017 cost
select z.zipcode, ifnull(cast(zl.listprice as int),z.[2017-06]) AS [2019_Cost]
from zillow z
left join zillow_list zl on z.zipcode = zl.zipcode
UNION
select zl.zipcode,cast(zl.listprice as int)
from zillow_list zl
left join zillow z on z.zipcode = zl.zipcode
where z.zipcode IS NULL and zl.listprice IS NOT NULL
"""

zillow_new = pandasql.sqldf(query)
zillow_new

In [ ]:
query = """
select zipcode from zillow_new
INTERSECT
select zipcode from airbnb
"""

# 44 zip codes in common, up from 24 earlier. 
pandasql.sqldf(query)

### JOIN & FILTER

In [ ]:
# Similar to Part 1

query = """
SELECT
    a.zipcode as ZipCode,
    neighbourhood_group_cleansed AS Area,
    count(price) AS Properties,
    sum(number_of_reviews) AS Reviews,
    [2019_Cost] AS [Cost$],
    (365-AVG(availability_year))*100/365 AS [Occupancy%],  -- Occupancy is inverse of Availability
    (cast(AVG(price) * (365-AVG(availability_year)) as int))/100*100 AS [Revenue$/Year],
    round([2019_Cost]/(AVG(price) * 365 * .75),2) AS BreakEven_Years,
    
    -- Plotting Data
    avg(latitude) AS latitude,
    avg(longitude) AS longitude,
    'ZipCode : ' || a.zipcode || ' . Area : ' || neighbourhood_group_cleansed AS text
FROM
    airbnb a
JOIN
    zillow_new z ON a.zipcode = z.zipcode
WHERE 1=1
    AND a.number_of_reviews > 1
    AND a.last_review > date('now','-1 years')
    --AND a.availability_365 not in (0,365)
GROUP BY
    a.zipcode
HAVING
    Properties > 1
ORDER BY
    BreakEven_Years
"""

az = pandasql.sqldf(query)

In [ ]:
# Interactive Grid
qgrid.show_grid(az.loc[:, 'ZipCode':'BreakEven_Years'])

# Fiddle around with Filters below

In [ ]:
# Plotly offline mode. No need for API token
plotly.offline.init_notebook_mode()

### FANCY


In [ ]:
az.sort_values(by=['Cost$'],inplace=True)
fig = go.Figure(
    [go.Bar(name='Total Cost', x=az['ZipCode'], y=az['Cost$']),
     go.Bar(name='Yearly Revenue', x=az['ZipCode'], y=az['Revenue$/Year']),
    ],
    go.Layout(barmode='stack',
              xaxis=dict(type='category',title='Zip Code'),
              yaxis=dict(title='Cost & Revenue'),
              title = 'Cost-Revenue')
)

fig.show()

In [ ]:
fig = px.scatter(az, x="Revenue$/Year", y="BreakEven_Years", color="Properties", size="Cost$",
                             hover_data=['Cost$','Area','ZipCode'],
                             title='ROI Quadrant : Bottom-Right - High Revenue & Quick BreakEven')
fig.show()

#### TODO : Add quadrant dividing lines, labels and column filters in plot

In [ ]:

fig = px.scatter_geo(az, lat="latitude",lon="longitude", size="Properties",
                     scope="usa", color = "BreakEven_Years", hover_data=['Cost$','Revenue$/Year','Area','ZipCode'])
fig.show()


### COMMERCIAL (FREE TIER) PLOTS

#### 1 . MAPBOX

In [ ]:
px.set_mapbox_access_token('pk.eyJ1Ijoic3FsZGJhIiwiYSI6ImNqeXg2OG1sZzBqczYzZG83N2FncGVobDIifQ.7yljGwjF4bJnx9JHaa1BIg')

fig = px.scatter_mapbox(az, lat="latitude", lon="longitude",  color="BreakEven_Years", size="Properties",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=25, zoom=10)
fig.show()

#### 2 . TABLEAU

Interactive, Filter-able map plot:  
https://public.tableau.com/views/cap1airbnbzillow/Invest  
(Private-Visibilty)

***

## SUMMARY

#### A data product was created to find profitable rental investment locations in New York city.

#### Assumptions :

1. Rent Price and Home Value remain constant.
3. Additional fees revenue and Routine maintenance cost ignored

#### Data Cleaning :
1. AirBnB missing zip codes calculated from latitude/longitude data
2. Additional Zillow zip codes data obtained from website

#### Data Munging :
1. New metric 'Availability' created as the average of 30,60,90 & 365 availability
2. Yearly revenue calculated using daily rent price, and 'averaged' by zip code to match 'median' cost data

#### Data Visualization :
1. Interactive grid containing profit metrics displayed for user self-service
2. Interactive box, scatter and map plots created for easy visual analysis


#### Recommendation:
1. zip codes 10025 & 10022 are good investment locations in a high-end established market like Manhattan, while 10021 is not.
2. zip codes 11434 & 11375 are good investment locations in a low-end new market like Queens, while 11374 is not

#### Future:
1. Look at TODO items
2. Look into regulatory issues related to housing & rent-control

#### References:
1. https://www.zillow.com/research/data/
2. https://www.redfin.com/blog/data-center/
3. http://insideairbnb.com/get-the-data.html
4. http://tomslee.net/category/airbnb-data

d-_-b